# Transfer Learning for Image Classification using PyTorch and the TLK API

This notebook uses the `tlk` library to do transfer learning for image classfication with a PyTorch pretrained model.

## 1. Import dependencies and setup parameters

This notebook assumes that you have already followed the instructions to setup a PyTorch environment with all the dependencies required to run the notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import PIL.Image as Image
import torch, torchvision
import requests
from io import BytesIO

# tlk imports
from tlk.datasets import dataset_factory
from tlk.models import model_factory
from tlk.utils.file_utils import download_and_extract_tar_file, download_file

# Specify a directory for the dataset to be downloaded
dataset_dir = os.environ["DATASET_DIR"] if "DATASET_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "dataset")
     
# Specify a directory for output
output_dir = os.environ["OUTPUT_DIR"] if "OUTPUT_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "output")

print("Dataset directory:", dataset_dir)
print("Output directory:", output_dir)

## 2. Get the model

In this step, we call the TLK model factory to list supported PyTorch image classification models. This is a list of pretrained models from [torchvision](https://pytorch.org/vision/stable/datasets.html) that we tested with our API. Optionally, the `verbose=True` argument can be added to the `print_supported_models` function call to get more information about each model (such as the classification layer, image size, the original dataset, etc).

In [ ]:
# See a list of available models
model_factory.print_supported_models(use_case='image_classification', framework='pytorch')

Next, use the TLK model factory to get one of the models listed in the previous cell. The `get_model` function returns a TLK model object that will later be used for training.

In [ ]:
model = model_factory.get_model(model_name='efficientnet_b0', framework='pytorch')

print("Model name:", model.model_name)
print("Framework:", model.framework)
print("Use case:", model.use_case)
print("Image size:", model.image_size)

## 3. Get the dataset

### Option A: Use your own dataset

To use your own image dataset for transfer learning with the rest of this notebook, format your images as `.jpg` files and save them in folders named after the classes that you want the model to predict. To provide a working example using the correct layout, we will download a flower species dataset. After downloading and extracting, you will have the following  subdirectories in your dataset directory. Each species subfolder will contain numerous `.jpg` files:

```
flower_photos
  └── daisy
  └── dandelion
  └── roses
  └── sunflowers
  └── tulips
```

When using your own dataset, ensure that it is similarly organized with folders for each class. Change the `custom_dataset_path` variable to point to your dataset folder.

In [ ]:
# For demonstration purposes, we download a flowers dataset. To instead use your own dataset, set the
# custom_dataset_path to point to your dataset's directory and comment out the download_and_extract_tar_file line.
custom_dataset_path = os.path.join(dataset_dir, "flower_photos")

if not os.path.exists(custom_dataset_path):
    download_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
    download_and_extract_tar_file(download_url, dataset_dir)

Call the dataset factory to load the dataset from the directory.

In [ ]:
# Load the dataset from the custom dataset path
dataset = dataset_factory.load_dataset(dataset_dir=custom_dataset_path,
                                       use_case='image_classification', 
                                       framework='pytorch')

print("Class names:", str(dataset.class_names))

Skip to the next step [4. Prepare the dataset](#4.-Prepare-the-dataset) to continue using the custom dataset.

### Option B: Use a dataset from the PyTorch's torchvision Datasets catalog

To use a torchvision dataset, specify the name of the dataset in the `get_dataset` function. This example uses the `CIFAR10` dataset from the [torchvision datasets for image classification](https://pytorch.org/vision/stable/datasets.html#image-classification), but you can choose from a variety of options. If the dataset is not found in the dataset directory it will be downloaded. Subsequent runs will reuse the already downloaded dataset.

These torchvision datasets are currently supported in the API:
* CIFAR10
* CIFAR100
* Country211
* DTD
* Food101
* FGVCAircraft
* RenderedSST2

In [ ]:
dataset = dataset_factory.get_dataset(dataset_dir=dataset_dir,
                                      use_case='image_classification', 
                                      framework='pytorch',
                                      dataset_name='CIFAR10',
                                      dataset_catalog='torchvision')

print(dataset.info)

print("Class names:", str(dataset.class_names))

## 4. Prepare the dataset

Once you have your dataset from Option A or Option B above, use the following cells to preprocess the dataset. We resize the images to match the selected models and batch the images, then split them into training and validation subsets.

In [ ]:
# Preprocess the dataset with an image size that matches the model and a batch size of 32
batch_size = 32
dataset.preprocess(model.image_size, batch_size=batch_size)

In [ ]:
# Split the dataset into training and validation subsets
dataset.shuffle_split(train_pct=.75, val_pct=.25)

## 5. Predict using the original model

We get a single batch from our dataset, and use that to call predict on our model. Since we haven't done any training on the model yet, it will give us predictions using the original ImageNet trained model.

In [ ]:
# Get a single batch from the dataset
images, labels = dataset.get_batch()
labels = [dataset.class_names[id] for id in labels]

# Download the ImageNet labels and load them into a list
labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
labels_file_path = os.path.join(dataset_dir, os.path.basename(labels_file))

if not os.path.exists(labels_file):
    download_file(labels_file, dataset_dir)

with open(labels_file_path) as f:
    imagenet_labels = f.readlines()
    imagenet_classes = [l.strip() for l in imagenet_labels]
    
# Predict using the original model
predictions = model.predict(images)
predictions = [imagenet_classes[id] for id in predictions]

In [ ]:
# Display the images with the predicted ImageNet label
plt.figure(figsize=(18,14))
plt.subplots_adjust(hspace=0.5)
for n in range(min(batch_size, 30)):
    plt.subplot(6,5,n+1)
    inp = images[n]
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    correct_prediction = labels[n] == predictions[n]
    color = "darkgreen" if correct_prediction else "crimson"
    title = predictions[n].title() if correct_prediction else "{}\n({})".format(predictions[n], labels[n]) 
    plt.title(title, fontsize=14, color=color)
    plt.axis('off')
_ = plt.suptitle("ImageNet predictions", fontsize=20)
plt.show()

print("Correct predictions are shown in green")
print("Incorrect predictions are shown in red with the actual label in parenthesis")

## 6. Transfer Learning

This step calls the TLK model's train function with the dataset that was just prepared. The training function will get the torchvision feature vector and add on a dense layer based on the number of classes in the dataset. The model is then compiled and trained based on the number of epochs specified in the argument.

In [ ]:
history = model.train(dataset, output_dir=output_dir, epochs=1)

## 7. Evaluate

The next step shows how the model can be evaluated. The model's evaluate function returns a list of metrics calculated from the dataset's validation subset.

In [ ]:
metrics = model.evaluate(dataset)

## 8. Predict

Lastly, we predict using the same single batch that we used earlier with the ImageNet trained model to visualize the model's predictions after training.

In [ ]:
# Predict with a single batch
predictions = model.predict(images)

# Map the predicted ids to the class names
predictions = [dataset.class_names[id] for id in predictions]

# Display the results
plt.figure(figsize=(16,16))
plt.subplots_adjust(hspace=0.5)
for n in range(min(batch_size, 30)):
    plt.subplot(6,5,n+1)
    inp = images[n]
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    correct_prediction = labels[n] == predictions[n]
    color = "darkgreen" if correct_prediction else "crimson"
    title = predictions[n].title() if correct_prediction else "{}\n({})".format(predictions[n], labels[n]) 
    plt.title(title, fontsize=14, color=color)
    plt.axis('off')
_ = plt.suptitle("Model predictions", fontsize=16)
plt.show()
print("Correct predictions are shown in green")
print("Incorrect predictions are shown in red with the actual label in parenthesis")

We can also predict using a single image that wasn't part of our original dataset. We download a flower image from the [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html) and then resize it to match our model.

In [ ]:
# Download an image from the web and resize it to match our model
image_url = 'https://c8.staticflickr.com/8/7095/7210797228_c7fe51c3cb_z.jpg'

image_shape = (model.image_size, model.image_size)
daisy = Image.open(BytesIO(requests.get(image_url).content)).resize(image_shape)
daisy

Then, we call predict by passing the np array for our image and add a dimension to our array to represent the batch.

In [ ]:
# Get the image as a np array and call predict while adding a batch dimension (with np.newaxis) 
daisy = np.array(daisy)/255.0
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
daisy = std * daisy + mean
daisy = np.clip(daisy, 0, 1)

result = model.predict(torch.Tensor(daisy[np.newaxis, ...]))

# Print the predicted class name
print(dataset.class_names[result[0]])

## 9. Export

Lastly, we can call the TLK model `export` function to generate a `saved_model.pb`. Each time the model is exported, a new numbered directory is created, which allows serving to pick up the latest model. 

In [ ]:
model.export(output_dir)

## Dataset Citations

```
@ONLINE {tfflowers,
author = "The TensorFlow Team",
title = "Flowers",
month = "jan",
year = "2019",
url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }

@ONLINE {CIFAR10,
author = "Alex Krizhevsky",
title = "CIFAR-10",
year = "2009",
url = "http://www.cs.toronto.edu/~kriz/cifar.html" }

@article{openimages,
  title={OpenImages: A public dataset for large-scale multi-label and multi-class image classification.},
  author={Krasin, Ivan and Duerig, Tom and Alldrin, Neil and Veit, Andreas and Abu-El-Haija, Sami
    and Belongie, Serge and Cai, David and Feng, Zheyun and Ferrari, Vittorio and Gomes, Victor
    and Gupta, Abhinav and Narayanan, Dhyanesh and Sun, Chen and Chechik, Gal and Murphy, Kevin},
  journal={Dataset available from https://github.com/openimages},
  year={2016}
}
```